In [2]:
# add path (for local)
import sys
repo_root = "../"
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

In [3]:
import optuna
from rdkit import RDLogger
from utils import generator_from_conf, conf_from_yaml
RDLogger.DisableLog('rdApp.*')

yaml_path = "config/optuna_generation.yaml"

conf = conf_from_yaml(yaml_path, repo_root)
name = conf.get("study_name")
n_trials = conf.get("n_trials")

def objective(trial):
    conf = conf_from_yaml(yaml_path, repo_root)
    # conf.setdefault("transition_args", {})
    # conf["transition_args"]["model_dir"] = "model/smiles/" + trial.suggest_categorical("model", ["drugs_zinc_gru", "drugs_zinc_lstm", "tf25_ported"])
    # conf["transition_args"]["sharpness"] = trial.suggest_float("sharpness", 0.7, 1.3)
    # conf["transition_args"]["top_p"] = trial.suggest_float("top_p", 0.994, 0.998)
    
    conf["policy_class"] = "UCTAP"
    conf.setdefault("policy_args", {})
    conf["policy_args"]["c"] = trial.suggest_float("c", 0.05, 0.4)
    conf["policy_args"]["best_rate"] = trial.suggest_float("best_rate", 0, 1)
    conf["policy_args"]["c_action"] = trial.suggest_float("c_action", 0, 0.4)
    conf["policy_args"]["prior_offset"] = trial.suggest_float("prior_offset", 0, 0.2)

    # conf.setdefault("generator_args", {})
    # conf["generator_args"]["eval_width"] = trial.suggest_int("eval_width", 1, 40)
    # # conf["generator_args"]["allow_rollout_overlaps"] = trial.suggest_categorical("allow_rollout_overlaps", [True, False])
    # conf["generator_args"]["n_evals"] = trial.suggest_int("n_evals", 1, 10)
    # conf["generator_args"]["n_tries"] = trial.suggest_int("n_tries", 1, 3)
    # conf["generator_args"]["filter_reward"] = trial.suggest_categorical("filter_reward", ["ignore", 0])
    # conf["generator_args"]["failed_parent_reward"] = trial.suggest_categorical("failed_parent_reward", ["ignore", -1])
    # conf["generator_args"]["terminal_reward"] = trial.suggest_categorical("terminal_reward", ["ignore", -1])
    # conf["generator_args"]["cut_failed_child"] = trial.suggest_categorical("cut_failed_child", [True, False])
    
    generator = generator_from_conf(conf, repo_root)
    generator.logger.info(f"params={trial.params}")
    
    max_generations, time_limit = conf.get("max_generations"), conf.get("time_limit")
    n_steps, best_reward_rate= conf.get("n_steps"), conf.get("best_reward_rate")
    
    for i in range(0, n_steps):
        generator.generate(max_generations=max_generations / n_steps, time_limit=time_limit / n_steps)
        average_reward = generator.average_reward()
        intermediate_value = (1 - best_reward_rate) * average_reward + best_reward_rate * generator.best_reward
        trial.report(intermediate_value, i)
        if trial.should_prune():
            print(f"Trial {trial.number} - Step {i}: average_reward={average_reward:.3f}, best_reward={generator.best_reward:.3f}, params={trial.params}")
            raise optuna.TrialPruned()

    trial.set_user_attr("average_reward", average_reward)
    trial.set_user_attr("best_reward", generator.best_reward)
    print(f"Trial {trial.number}: average_reward={average_reward:.3f}, best_reward={generator.best_reward:.3f}")
    generator.analyze()
    generator.plot(**conf.get("plot_args"))
    return (1 - best_reward_rate) * average_reward + best_reward_rate * generator.best_reward
    
def print_trial(trial: optuna.Trial):
    print(f"Trial {trial.number} score={trial.value:.3f}, attrs={trial.user_attrs}, params={trial.params}")
    
def print_best_trials(study: optuna.Study):
    print("Optuna trials completed.")
    print("------ Best trials -----")
    best_trials = sorted([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE], key=lambda t: t.value, reverse=True)[:5]
    for t in best_trials:
        print_trial(t)

/home/fujii/anaconda3/envs/v3-forge/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterHierarchyMatcher> already registered; second conversion method ignored.


In [ ]:
# start search
storage = "sqlite:///optuna/" + name + ".db"
sampler = sampler=optuna.samplers.TPESampler(multivariate=True, group=True)
# sampler = optuna.samplers.GPSampler(deterministic_objective=False) # better if not using pruner?
pruner = optuna.pruners.MedianPruner(n_startup_trials=3, n_warmup_steps=0, interval_steps=1)
study = optuna.create_study(direction="maximize", study_name=name, storage=storage, sampler=sampler, pruner=pruner)
# study.enqueue_trial({"model": "drugs_zinc_gru", "sharpness": 1.0, "top_p": 0.995, "c": 0.2, "best_rate": 0.5, "use_prior": True, "prior": 0.6, "eval_width": 1, "n_evals": 1, "n_tries": 1, "filter_reward": 0, "failed_parent_reward": "ignore", "terminal_reward": "ignore", "cut_failed_child": False})
study.optimize(objective, n_trials=n_trials)
print_best_trials(study)

/home/fujii/anaconda3/envs/v3-forge/lib/python3.11/site-packages/optuna/_experimental.py:32: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
/home/fujii/anaconda3/envs/v3-forge/lib/python3.11/site-packages/optuna/_experimental.py:32: ExperimentalWarning: Argument ``group`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-07-09 14:07:58,417] A new study created in RDB with name: smirks_200000


In [ ]:
# continue search
study = optuna.study.load_study(study_name="d_score_200000", storage="sqlite:///optuna/d_score_200000.db")
study.optimize(objective, n_trials=300)
print_best_trials(study)

In [ ]:
# add parameters
study = optuna.study.load_study(study_name="d_score_200000", storage="sqlite:///sqlite:///optuna/d_score_200000.db")

sampler = sampler=optuna.samplers.TPESampler(multivariate=True, group=True)
pruner = optuna.pruners.MedianPruner(n_startup_trials=3, n_warmup_steps=0, interval_steps=1)
new_storage="sqlite:///optuna/d_score_200000_new.db"
study_with_new_param = optuna.create_study(direction="maximize", study_name=name, storage=new_storage, sampler=sampler, pruner=pruner)

for trial in study.trials:
    params = trial.params
    dists = trial.distributions

    params["cut_failed_child"] = False
    dists["cut_failed_child"] = optuna.distributions.CategoricalDistribution([True, False])

    trial.params = params
    trial.distributions = dists

    study_with_new_param.add_trial(trial)